In [ ]:
#imports
import os
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Normalize
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from tqdm import tqdm as tqdm
import torch.optim.lr_scheduler as lr_scheduler

In [ ]:
TRAIN_DIRECTORY = 'E:/Ishara augmented notebook/Ishara augmented data split/train'
VAL_DIRECTORY = 'E:/Ishara augmented notebook/Ishara augmented data split/val'
TEST_DIRECTORY = 'E:/Ishara augmented notebook/Ishara augmented data split/test'

In [ ]:
print(os.listdir(TRAIN_DIRECTORY))

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Hyper-parameters 
num_epochs = 15
batch_size = 100
learning_rate = 0.01

In [ ]:
img_transform = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()])
train_data = ImageFolder(TRAIN_DIRECTORY, transform=img_transform)
val_data = ImageFolder(VAL_DIRECTORY, transform=img_transform)
test_data = ImageFolder(TEST_DIRECTORY, transform=img_transform)

In [ ]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

In [ ]:
train_dl = DataLoader(train_data, batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_dl = DataLoader(val_data, batch_size*2, num_workers=4, pin_memory=True)
val_dl = DataLoader(test_data, batch_size*2,shuffle=True, num_workers=4, pin_memory=True)

In [ ]:
#load pretrained model
#model = torchvision.models.vgg16_bn(pretrained = True)

import torchvision.models as models
model = models.resnet18(pretrained=True)
# model = models.alexnet(pretrained=True)
# model = models.squeezenet1_0(pretrained=True)
# model = models.vgg16(pretrained=True)
# model = models.densenet161(pretrained=True)
# model = models.inception_v3(pretrained=True)
# model = models.googlenet(pretrained=True)
# model = models.shufflenet_v2_x1_0(pretrained=True)
# model = models.mobilenet_v2(pretrained=True)
# model = models.resnext50_32x4d(pretrained=True)
# model = models.wide_resnet50_2(pretrained=True)
# model = models.mnasnet1_0(pretrained=True)

In [ ]:
for param in model.parameters():
    param.requires_grad = True

model.classifier = nn.Sequential(nn.Linear(in_features=1000, out_features=256, bias=True),
                                 nn.ReLU(inplace=True),
                                 nn.BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True),
                                 nn.Linear(in_features=256, out_features=128, bias=True),
                                 nn.ReLU(inplace=True),
                                 nn.BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True),
                                 nn.Linear(in_features=128, out_features=35, bias=True))


model.to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
n_total_steps = len(train_dl)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_dl):
    
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

print('Finished Training')

In [ ]:
def check_acc(dl):
    n_correct = 0
    n_samples = 0
    model.eval()

    with torch.no_grad():
        for images, labels in dl:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            # max returns (value ,index)
            _, predicted = torch.max(outputs, 1)

            n_correct += (predicted == labels).sum()
            n_samples += predicted.size(0)

        print(f"acc : Got {n_correct} / {n_samples} with accuracy {float(n_correct)/float(n_samples)*100:.2f}")

In [ ]:
check_acc(train_dl)

In [ ]:
check_acc(test_dl)

In [ ]:
check_acc(val_dl)